### Defining Custom Functions

Although apricot has several feature-based and graph-based submodular functions implemented, the full space of submodular functions is too large to be stored on GitHub. Fortunately, for those who want to optimize their own functions, the apricot makes it easy to define custom functions and optimize them using the built-in optimizers. 

This tutorial will cover two ways of optimizing custom functions using apricot: (1) writing a function that simply returns the quality of any set of examples that is passed in and wrapping it with a custom selection objection, or (3) writing a class in the style of the existing selectors. The first approach is likely faster to implement but slower to execute, because a key aspect of the selection objects is using cached statistics to speed up calculations. Regardless of the approach taken, the existing optimizers can be applied to either.

In [1]:
%pylab inline
numpy.random.seed(0)

Populating the interactive namespace from numpy and matplotlib


Let's start off with the simplest way to optimize a custom function: by defining a set function and passing it in to `CustomSelection` or `CustomGraphSelection`. These selectors encode all the functionality needed for performing selection, and are initialized in the same way as the built-in selector objects, except that the user also passes in the function you want optimized. 

The custom function should take in a matrix where each row is an example in the selected set and each column is an attribute about that example. If the function is feature-based, each column should be a feature (and `CustomSelection` should be used); if the function is graph-based, each column should be the similarity to an example in the ground set and there should be a number of columns equal to the number of elements in the ground set (and `CustomGraphSelection` should be used).

### Custom Feature-Based Selection

The `CustomSelection` object serves as a wrapper around a feature-based function. A function is "feature-based" when it relies on the feature values of an example rather than the similarity with other examples. The practical importance of this distinction is that, usually, feature-based functions can be optimized efficiently and scale to massive data sets because they do not require storing a similarity matrix that is quadratic with the number of examples.

The built-in `FeatureBasedSelection` and `MaxCoverageSelection` objects wrap examples of feature-based functions. To demonstrate that the custom selection object works, let's implement a custom version of the function implemented in `FeatureBasedSelection` and compare it against the built-in object.

Here is the submodular function:

In [2]:
def sqrt_feature_based_function(X):
    return numpy.sqrt(X.sum(axis=0)).sum()

We can wrap our function using the `CustomSelection` object.

In [3]:
from apricot import CustomSelection

model = CustomSelection(100, sqrt_feature_based_function, optimizer='naive', verbose=True)

Now let's create a data set of random non-negative values to apply our method to.

In [4]:
X = numpy.exp(numpy.random.randn(10000, 100))

Now, let's compare against the built-in version.

In [5]:
from apricot import FeatureBasedSelection

model = CustomSelection(100, sqrt_feature_based_function, optimizer='naive')
model.fit(X)

model0 = FeatureBasedSelection(100, 'sqrt', optimizer='naive')
model0.fit(X)

numpy.all(model.ranking == model0.ranking), (model.gains - model0.gains).sum()

(True, -2.0747847884194925e-12)

Great! It looks like both implementations are selecting the same examples and reporting the same gain.   

Now, let's make sure that we get the same subset even when we use a more complicated optimization method (be sure to set the same random state when using an optimizer that uses random values).

In [6]:
model = CustomSelection(100, sqrt_feature_based_function, optimizer='stochastic', random_state=0)
model.fit(X)

model0 = FeatureBasedSelection(100, 'sqrt', optimizer='stochastic', random_state=0)
model0.fit(X)

numpy.all(model.ranking == model0.ranking), (model.gains - model0.gains).sum()

(True, -3.979039320256561e-13)

It looks like we're getting the same subset even with a stochastic optimizer.

Now, let's look at timing. How much faster is the built-in feature-based function selection than the custom selector?

In [7]:
model1 = CustomSelection(100, sqrt_feature_based_function, optimizer='naive')
model2 = FeatureBasedSelection(100, 'sqrt', optimizer='naive')

%timeit model1.fit(X)
%timeit model2.fit(X)

7.56 s ± 40.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.26 s ± 5.43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Looks like the built-in implementation is much faster than our custom function implementation. In more thorough evaluations not shown here, it takes roughly 1s for `FeatureBasedSelection` to get initialized, and that increasing the number of selected examples or the size of the data set does not have much of an effect. In contrast, the `CustomSelection` object scales linearly with the number of examples selected and with the size of the data set. 

Although the previous example focused on recreating the existing feature-based selector, the real benefit of using the `CustomSelector` is that any submodular function can be passed in and optimized. Let's build a weird function that is a mixture of two step functions. To ensure that this function is submodular and that optimization is proceeding reasonably, let's compare the results we get from using the naive greedy algorithm to the lazy greedy algorithm. When the underlying function is not submodular the two will diverge, with the naive greedy algorithm producing the correct results.

In [8]:
def weird_function(X):
    X1, X2 = X > 1, X < 2
    return numpy.sqrt((X * X1).sum(axis=0)).sum() + numpy.log1p((X2 * X).sum(axis=0)).sum() 

model1 = CustomSelection(100, weird_function, optimizer='naive')
model2 = CustomSelection(100, weird_function, optimizer='lazy')

model1.fit(X)
model2.fit(X)

numpy.all(model1.ranking == model2.ranking), (model1.gains - model2.gains).sum()

(True, 0.0)

Great! Both optimizers are producing the same results even though they are optimizing this weird function.

One can also conveniently pass in parameters to the custom function as a dictionary to the `function_kwds` parameter in the selector. This dictionary gets passed to the function as keyword arguments, and so the keys in the dictionary should correspond to the parameter names in the custom function that is passed in.  

Let's see the use of passing parameters through the selector in the context of making a custom function that is a mixture of two objectives with weights that can be tuned. We can compare the results of this to the built-in mixture selector that offers the same functionality.

In [9]:
from apricot import MixtureSelection

def mixture_function(X, w0=1.0, w1=1.0):
    X_sum = X.sum(axis=0)
    return w0 * numpy.sqrt(X_sum).sum() + w1 * numpy.log1p(X_sum).sum()

model1 = CustomSelection(100, mixture_function, function_kwds={'w0': 0.3, 'w1': 1.8}, optimizer='naive')
model2 = MixtureSelection(100, [FeatureBasedSelection(100, 'sqrt'), FeatureBasedSelection(100, 'log')],
                        weights=[0.3, 1.8])

model1.fit(X)
model2.fit(X)

numpy.all(model1.ranking == model2.ranking), (model1.gains - model2.gains).sum()

(True, -8.752998326144734e-13)

### Custom Graph-Based Selection

Arguably graph-based functions, such as facility location and graph cut, are much more popular in the literature. These functions differ from feature-based functions in that the optimization proceeds over similarities between examples rather than the feature-values themselves. Indeed, one need not even have featurized examples at all: for instance, if your examples are locations and the "similarity" is derived from the physical distance between the locations, there is no need to even have underlying features (though for this example the underlying features might be the long/lat coordinates of the locations) (See Example B: Airport Selection for a more in-depth example).

Our graph-based function should be implemented in the same general manner as a feature-based function. The function will take in a set of potential selected items and return the quality of that set. However, this function will assume that the columns on this data set will be the similarities between the example and all examples in the ground set. For example, here is the facility location objective function.

In [10]:
def facility_location_function(X):
    return X.max(axis=0).sum()

All we have to do is pop this function into a `CustomGraphSelection` object. Note that we do not need to specify a method for turning data sets into similarity matrices. In the same way that the built-in graph-based functions can handle this conversion internally (and have a parameter `metric` that defines how similarities are calculated), the `CustomGraphSelection` object can also handle this conversion internally. If you want to pass in a precomputed similarity matrix, you can specify `metric='precomputed'`.

Here, we will use the lazy optimizer because it provides massive speed boosts over the naive optimizer on graph-based functions.

In [11]:
from apricot import CustomGraphSelection

model = CustomGraphSelection(100, facility_location_function, optimizer='lazy', verbose=True)

Let's compare against the built-in facility location selector to ensure the implementation is correct.

In [12]:
from apricot import FacilityLocationSelection

model = CustomGraphSelection(100, facility_location_function, metric='corr', optimizer='lazy')
model.fit(X)

model0 = FacilityLocationSelection(100, metric='corr', optimizer='lazy')
model0.fit(X)

numpy.all(model.ranking == model0.ranking), (model.gains - model0.gains).sum()

(True, -1.8474111129762605e-13)

All of the same cool functionality we showed for feature-based functions with `CustomSelection` is also possible with graph-based functions and `CustomGraphSelection`.

### Writing your own selection objects

Although the `CustomSelection` and `CustomGraphSelection` wrappers are simple to use, they are likely much slower than using the built-in selectors. A reason for this is that the functions themselves are sped up using numba, making them significantly faster and multi-thread parallel. However, even if users wrote their own numba functions and wrapped them in the selection wrappers (which is certainly possible), the built-in selectors would likely still be faster. This is because the built-in selectors are able to cache function-specific statistics about the stored value at each iteration which can be used to speed up the calculation. 

For example, evaluating a feature-based function involve calculating the sum of each column of the selected set, applying a concave function, and then summing the result. However, if we already knew the sum of each column from the selected set in the previous iteration, we could speed up the computation significantly by simply adding the new element to these stored values element-wise.

Likewise, evaluating a facility-location function involves calculating the nearest selected example for each example in the ground set. If you cache the similarity between each example in the ground set and its nearest selected example, you simply need to see whether the proposed newest item is more similar than the currently stored most-similar example using an element-wise max operation.

Because the wrapped custom functions only evaluate the quality of a set, they do not even know which items were ultimately selected. Unfortunately, the set of statistics that would be valuable to cache depends on the objective function itself, and so there is no single set of statistics that could be passed into any function.

If the wrapped selectors are not fast enough, one may choose to implement their own selector from scratch. Going this route allows the user to both define the function that they want to optimize as well as the statistics that should be cached.

Below is an example of the basic skeleton that needs to be filled in to define an entire selector.

In [13]:
from apricot import BaseSelection

class SkeletonSelection(BaseSelection):
    # If you're defining a feature-based function you should inherit from BaseSelection
    # If you're defining a graph-based function and want to make use of the built-in
    # similarity calculations, you should inherit from BaseGraphSelection, which itself
    # inherits from BaseSelection.
    
    def __init__(self, n_samples, initial_subset=None, optimizer='two-stage', 
        optimizer_kwds={}, n_jobs=1, random_state=None, 
        verbose=False):
        
        # If defining a graph-based function, you should also include a metric
        # parameter, with the default in apricot being "metric='euclidean'".
        # You will also need to pass the metric into the call to `super` below
        # by adding "metric=metric" anywhere.
        
        # This function must call `__init__` of the base class, as below. 
        # Other custom arguments, such as hyperparameters for the underlying
        # submodular function, should be stored in this method. 
        
        super(SkeletonSelection, self).__init__(n_samples=n_samples, 
            initial_subset=initial_subset, optimizer=optimizer, 
            optimizer_kwds=optimizer_kwds, n_jobs=n_jobs, random_state=random_state, 
            verbose=verbose)

    def _initialize(self, X):
        # This function includes any logic that should be executed before the
        # selection process begins, such as initializing cached values and
        # setting them to values from the initial subset.
        
        super(SkeletonSelection, self)._initialize(X)

    def _calculate_gains(self, X, idxs=None):
        # This function returns the gain in the objective function that each
        # item in X[idxs] would return. The returned vector should be of
        # length len(idxs) or len(self.idxs) if idxs is None. The value of
        # gains[i] should be the gain associated with adding X[idxs[i]] to
        # the selected set.

        idxs = idxs if idxs is not None else self.idxs
        gains = numpy.zeros(idxs.shape[0], dtype='float64')
        return gains

    def _select_next(self, X, gain, idx):
        # This function takes in a single example, X, the corresponding gain
        # of adding this element to the selected set, and the index of this
        # example in the full data set, and updates the cached values accordingly.

        super(SkeletonSelection, self)._select_next(X, gain, idx)

Each of the methods in the above class must be filled out, but likely the most important method is `_calculate_gains`. Notably, this method DOES NOT simply return the quality of some subset `X`, as the functions we wrapped before did. Rather, this method returns the gain that each element in `X[idxs]` would provide if it were added as the next item. Thus, each iteration of selection using the naive optimizer involves a single call to `_calculate_gains` followed by choosing the element with the largest gain. This calculates the gains for only those items indexes in `idxs` because there is no need to calculate the gain for elements that have already been selected, and also because some optimization strategies involve sampling only a portion of the data set to evaluate.

Let's see an example of a simple implementation of a feature-based function.

In [14]:
from apricot import BaseSelection

class SqrtFeatureBasedSelector(BaseSelection):
    '''This is a minimal implementation of a feature-based function with a sqrt.'''
    
    def __init__(self, n_samples, initial_subset=None, optimizer='two-stage', 
        optimizer_kwds={}, n_jobs=1, random_state=None, 
        verbose=False):
        
        super(SqrtFeatureBasedSelector, self).__init__(n_samples=n_samples, 
            initial_subset=initial_subset, optimizer=optimizer, 
            optimizer_kwds=optimizer_kwds, n_jobs=n_jobs, random_state=random_state, 
            verbose=verbose)

    def _initialize(self, X):
        # The cached values will be the column sums.
        self.current_values = numpy.zeros(X.shape[1])
        
        # We should also keep track of the total gain thus far
        # so we can calculate the marginal gain of adding each
        # element quickly.
        self.total_gain = 0.0
        super(SqrtFeatureBasedSelector, self)._initialize(X)

    def _calculate_gains(self, X, idxs=None):
        # The gains are the increase in the objective. This can be
        # calculated as the objective value of each example minus
        # the stored accumulated gain. Given that this is trivially
        # vectorizable, the code is not actually complex.

        idxs = idxs if idxs is not None else self.idxs
        gains = numpy.sqrt(X[idxs] + self.current_values).sum(axis=1) - self.total_gain
        return gains

    def _select_next(self, X, gain, idx):
        # Because we are storing column sums we only need to do an
        # element-wise addition to update the cached values and
        # another addition to store the accumulated gain.
        
        self.current_values += X
        self.total_gain += gain
        super(SqrtFeatureBasedSelector, self)._select_next(X, gain, idx)

Let's compare the results from this custom object to the built-in feature-based selector and to the wrapped custom function.

In [15]:
model0 = FeatureBasedSelection(10, 'sqrt', optimizer='naive')
model0.fit(X)

model1 = CustomSelection(10, sqrt_feature_based_function, optimizer='naive')
model1.fit(X)

model2 = SqrtFeatureBasedSelector(10, optimizer='naive')
model2.fit(X)

print(model0.ranking, "\n", model0.gains, "\n")
print(model1.ranking, "\n", model1.gains, "\n")
print(model2.ranking, "\n", model2.gains, "\n")

[9403 9550 3391 1786 9858 5697 1647 5221 5379 1053] 
 [136.62059032  69.77395573  52.10092935  43.88411145  39.33852143
  35.93244774  32.49506686  30.3090869   27.89198382  25.90773747] 

[9403 9550 3391 1786 9858 5697 1647 5221 5379 1053] 
 [136.62059032  69.77395573  52.10092935  43.88411145  39.33852143
  35.93244774  32.49506686  30.3090869   27.89198382  25.90773747] 

[9403 9550 3391 1786 9858 5697 1647 5221 5379 1053] 
 [136.62059032  69.77395573  52.10092935  43.88411145  39.33852143
  35.93244774  32.49506686  30.3090869   27.89198382  25.90773747] 



Good news, it looks like all of them select the same elements and calculate the same gains for each one. 

Now, let's compare timings.

In [16]:
model0 = FeatureBasedSelection(100, 'sqrt', optimizer='naive')
%timeit model0.fit(X)

model1 = CustomSelection(100, sqrt_feature_based_function, optimizer='naive')
%timeit model1.fit(X)

model2 = SqrtFeatureBasedSelector(100, optimizer='naive')
%timeit model2.fit(X)

1.27 s ± 5.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
7.6 s ± 49.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
486 ms ± 1.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


As we saw earlier, the wrapper for custom functions is far slower than the built-in selector. Interestingly, it seems like the custom feature-based method we just implemented might be faster than the built-in selector. Does this hold up with larger data sets, or is it only the case for the somewhat small toy data set we were using throughout this notebook?

In [ ]:
X = numpy.exp(numpy.random.randn(100000, 500))

model0 = FeatureBasedSelection(1000, 'sqrt', optimizer='naive')
%timeit model0.fit(X)

model2 = SqrtFeatureBasedSelector(1000, optimizer='naive')
%timeit model2.fit(X)

25.8 s ± 692 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


It looks like the built-in selector has a small startup cost (around 1s), as we saw earlier in this tutorial, but ends up scaling better to larger problems.

### Conclusions

In this notebook we saw how one can use apricot to optimize custom functions in two convenient ways. The simplest way is to define a function that evaluates a set and wrap it with either `CustomSelection` or `CustomGraphSelection`, but an alternative that potentially can be significantly faster is to define an entire selection object. Either approach gives a user access to the wide variety of features implemented in apricot, such as optimizing their function using any predefined optimization algorithm.

Although the functions that we optimized here were entirely submodular, in keeping with the focus of apricot, the custom functions need not necessarily be. Recent work has suggested that supermodular functions can approximately optimized well, in practice, using the same naive greedy algorithm that are used to optimize submodular function. The naive greedy algorithm will no longer have the same theoretical guarantees when applied to supermodular functions instead of submodular functions, but usage in apricot is the same: define a custom function and wrap it, or define a selector that caches statistics to speed up computation.